# Project #1 Artificial intelligance
# Gandalf and Gray Allies Game
# (spring 2022)

    Amirmahdi Ansaripour

    Student ID: 810198358



##   Approach:

This assignment is about usage of informed and uninformed search algorthims in finding solutions for "Gandalf and Gray allies" game.

##   Descriptioin:

The game has some actors with different duties:

#### Gandalf: 

The main character that has to guide the Gray Allies to their safe castles and finally go to the last destination.
At each step, Gandalf can carry at most one ally.
He is not allowed to go to Orcs' houses, but can move in their under-controlled houses with alot of caution.


#### Orc(s): 

They're the strong enemies of Gandalf. Each Orc has a military degree n showing that cells which their Manhattan Distance is less than or equal to n are under conrtol of that Orc.

Each Orc is asleep initially and wakes up only if Gandalf or his friends walk through his under-controlled houses more than n times in a row.

#### Gray Allies: 
Gandalf friends that need his help to go to their safe castles.


## Importing Data and Modeling the game

At first, a text file is given as input that contains some data:

1.Rows and columns of the table

2.Gondalf initial and destination cell

3.Number of Orcs and allies

4.Each Orc's position and military degree

5.Each ally's initial position

6.Each ally's destination(in the same order as number 5)

## Modeling the table (Basic class)

Data is extracted and stored in a class named Basic.

Basic class contains some common and basic functions needed for all algorithms.

The class then makes the match table in the following way:

1.Orcs are displayed with '@'. Each Orc has an index (beginning with A) that under-controlled houses of each is filled with that index.

2.Allies are shown with numbers (beginning with 0) and each ally's destination is shown as '&' + ally's number

There is another field named 'enemy_strength', which is a dictionary that shows each Orc's military degree.

## Modeling each state

Each state in the game (including start and goal states) is a tuple and has some features:

1.Position: (x, y).

2.Enemy_area: -(no enemy in this cell),A,B,etc.

3.Allowed_movements: If Gondalf is currently in an Orc's controlled house, 
this feature shows how many steps he can take more in this region and the Orc won't wake up. 
If he isn't in such houses, this value is inf.

4.Taken_ally: If Gondalf has already taken an ally, this field equals the ally's number; Otherwise, it's -1. 

5.Allies_state: A string showing how many and which allies have been currently placed in their castles. 

## Start and Goal States

According to the previous step, features of start and goal states are:

Position: Given in the input file.

Enemy Area: For both is '-', regarding to the fact that the initial and goal houses aren't under control of any Orc.

Allowed_movements: For both is inf, exactly because of the Enemy Area reason.

Taken_ally: Obviously its '-1' for start state, and since all alies should be in their destinations by the end, it's '-1' for goal state too.

Allies_state: It's '0' * number of allies for start, since no ally has reached its destination yet. For goal state this feature is '1' * number of allies.


In [1]:
import math
import copy
import time
from _heapq import heappush, heappop

In [2]:
class Basic:
    def __init__(self,file_name):
        test_case = open(file_name)
        lines = []
        self.enemy_strength = dict()
        for data in test_case:
            lines.append(data.split())

        self.rows = int(lines[0][0])
        self.columns = int(lines[0][1])
        self.start_position = (int(lines[1][0]), int(lines[1][1]))
        self.goal_position =  (int(lines[2][0]), int(lines[2][1]))
        self.num_of_enemies = int(lines[3][0])
        self.num_of_allies = int(lines[3][1])
        self.table = [[['-'] for i in range(self.columns)] for j in range(self.rows)]
        self.enemy_strength['-'] = math.inf 
        self.put_enemies(lines)
        self.put_allies(lines)
        x = self.start_position[0]
        y = self.start_position[1]
        x_f = self.goal_position[0]
        y_f = self.goal_position[1]
        self.start_state = (self.start_position, self.table[x][y][0], math.inf, '-1', '0' * self.num_of_allies)
        self.goal_state = (self.goal_position, self.table[x_f][y_f][0], math.inf, '-1', '1' * self.num_of_allies)

    # based on military degree of an orc, show the under-controlled houses
    def put_enemy_strengths(self, length, enemy_index, x, y):
       for j in range(0, length + 1):
            x_min_range = max(0, y - length + j)
            x_max_range = min(y + length - j, self.columns - 1)
            for t in range(x_min_range, x_max_range + 1):
                if (x - j >= 0):
                    if(self.table[x - j][t][0] != '@'):    
                        self.table[x - j][t][0] = enemy_index
                if (x + j <= self.rows - 1):
                    if (self.table[x + j][t][0] != '@'):
                        self.table[x + j][t][0] = enemy_index

    


        #print(self.goal_state)
    def put_enemies(self, lines):
        enemy_index = 'A'
        for i in range(4, 4 + self.num_of_enemies):
            x = int(lines[i][0])
            y = int(lines[i][1])
            self.table[x][y][0] = '@'
            length = int(lines[i][2])
            self.put_enemy_strengths(length, enemy_index, x, y)
            self.enemy_strength[enemy_index] = length
            enemy_index = chr(ord(enemy_index) + 1)  
    

    # split the input node into some data
    def elicit_data(self, node):
        node_pos = node[0]
        x = node_pos[0]
        y = node_pos[1]
        node_enemy_area = node[1]
        node_ally_index = node[3]
        node_ally_string = node[4]
        return [node_pos, x, y, node_enemy_area, node_ally_index, node_ally_string]
                
        
    def put_allies(self, lines):
        ally_index = 0
        #Placing allies
        for i in range(4 + self.num_of_enemies, 4 + self.num_of_enemies + self.num_of_allies):
            x = int(lines[i][0])
            y = int(lines[i][1])
            self.table[x][y].append(str(ally_index))
            self.enemy_strength[str(ally_index)] = math.inf
            ally_index += 1

        ally_index = 0
        #Placing destinations of allies
        for i in range(4 + self.num_of_enemies + self.num_of_allies, len(lines)):
            x = int(lines[i][0])
            y = int(lines[i][1])
            self.table[x][y].append('&' + str(ally_index))
            self.enemy_strength['&' + str(ally_index)] = math.inf
            ally_index += 1
            
    def print_table(self):
        for row in self.table:
            print(row)
    
    
    def take_or_put_ally(self,child, node_ally_index, node_ally_string, x, y):
        # Put an ally
        for i in range(len(self.table[x][y])):
            if (self.table[x][y][i][0] == '&'):
                if (self.table[x][y][i][-1] == node_ally_index):
                    child[3] = '-1' # taken_ally
                    new_allies = copy.deepcopy(node_ally_string)
                    new_allies = new_allies[:int(node_ally_index)] + "1" + new_allies[int(node_ally_index) + 1:]
                    child[4] = new_allies # ally_string
            
        # Take an ally
        for j in range(len(self.table[x][y])):
            if (self.table[x][y][j].isdigit() == True):
                if (node_ally_index == '-1'):
                    ally_index = self.table[x][y][j]
                    child[3] = ally_index # ally_index
        return child
    
    
    #Used for showing the path
    def print_par(self, cur, string, parent):
        #print(str(cur[0]) + ":" + str(cur[2]))
        if cur == self.start_state :
            return string
        par_node = parent[cur]
        if(cur[0][0] == par_node[0][0] - 1):
            string += "U"
            #print("U", end = '')
        elif(cur[0][0] == par_node[0][0] + 1):
            string += "D"
            #print("D", end = '')
        elif(cur[0][1] == par_node[0][1] + 1):
            string += "R"
            #print("R", end = '')
        elif(cur[0][1] == par_node[0][1] - 1):
            string += "L"
            #print("L", end = '')
        return self.print_par(parent[cur], string, parent)
    
    def make_new_child(self, a, b, enemy_area, child):
        lst = []
        lst.append((a, b))
        lst.append(self.table[a][b][0])
        #print(enemy_area)
        #print(self.table[a][b][0])
        if(enemy_area != self.table[a][b][0]):
            lst.append(self.enemy_strength[self.table[a][b][0]])
        else:
            lst.append(child[2] - 1)
        lst.append(child[3])
        lst.append(child[4])
        lst = tuple(lst)
        return lst
    
    def end_search(self, node, parent, explored):
        print("Goal found !")
        path = self.print_par(node, "", parent)
        print("Path through start to goal: %s" %path[::-1])
        print("Length of the answer: %s" %len(path))
        print("Number of states seen: %s" %len(explored))

    

## BFS Algorithm
In BFS algorithm, we use a queue(frontier) and pop from it till it becomes empty.
At first, we add the start state. After that, in each step, we pop from the queue and explore the popped node. If it's the goal, the algorithm is ended. Else, some other nodes can be made based on that node (going to left, right, ... or taking and putting an ally).

We add those nodes to our frontier list if they're not repeated (in action_BFS function). We don't need to check and update the duplicated nodes costs since BFS proceeds level by level.

The 'if(lst[2] >= 1)' in all algorithms checks that if we're already in an Orc house and the next state results in Orc's waking up, we shouldn't add that state to frontier.

In [3]:
class BFS(Basic):
    def __init__(self, file_name):
        print("Running BFS algorithm on file %s" %file_name)
        super().__init__(file_name)
        self.parent = dict()
        self.explored = dict()
        self.frontier = []
        
    def action_BFS(self, lst, node_):
        if (not lst in self.explored):
            if (lst[2] >= 1):
                self.frontier.append(lst)
                self.parent[lst] = node_
                self.explored[lst] = True


    def run_BFS(self):
        self.parent[self.start_state] = -1
        self.frontier.append(self.start_state)
        #print(start_state)

        while(len(self.frontier) != 0):
            node = self.frontier.pop(0)
            self.explored[node] = True
            node_pos, x, y, node_enemy_area, node_ally_index, node_ally_string = self.elicit_data(node)

            # Break condition
            if node == self.goal_state:
                self.end_search(node, self.parent, self.explored)
                return True
    
            child = list(copy.deepcopy(node))
            # What you can do in this state
        
            child = self.take_or_put_ally(child, node_ally_index, node_ally_string, x, y)
            

            if (x - 1 >= 0):
                if (self.table[x - 1][y][0] != '@'):
                    lst = self.make_new_child(x - 1, y, node_enemy_area, child)
                    self.action_BFS(lst, node)
            
            if(y - 1 >= 0):
                if (self.table[x][y - 1][0] != '@'):
                    lst = self.make_new_child(x, y - 1, node_enemy_area, child)
                    self.action_BFS(lst, node)
                    
            if(x + 1 <= self.rows - 1):
                if(self.table[x + 1][y][0] != '@'):
                    lst = self.make_new_child(x + 1, y, node_enemy_area, child)
                    self.action_BFS(lst, node)
                    
            if(y + 1 <= self.columns - 1):
                if (self.table[x][y + 1][0] != '@'):
                    lst = self.make_new_child(x, y + 1, node_enemy_area, child)
                    self.action_BFS(lst, node)
                    
        return False

## IDS Algorithm
In IDS algorithm, we use the DFS but with a depth limitation (the 'max_depth' field in IDS class). We iteratively increment the depth (run_IDS function for loop); If the goal is met, we're done. Other wise, we keep incrementing the depth ('deepen' function).

In deepen function, we do the same process of run_BFS except that we use recursion to explore new nodes (DFS approach). 

What's more, in this algorithm, we should keep the distance (or cost) of each node from root and update it if we met a duplicated node with lower cost (second 'if' in action_IDS). The reason is obvious since we said that exploring nodes is depth-wise.  

In [4]:
class IDS(Basic):
    def __init__(self, file_name, depth):
        print("Running IDS algorithm on file %s" %file_name)
        super().__init__(file_name)
        self.distance = dict()
        self.parent = dict()
        self.explored = dict()
        self.max_depth = depth

    def run_IDS(self):
        found = False
        self.distance[self.start_state] = 0  # Distance of start from itself is zero
        for i in range(self.max_depth + 1):
            self.explored.clear()
            result = self.deepen(self.start_state, 0, i)
            if(result == True):
                print("Goal is found in depth %s !" %i)
                self.end_search(self.goal_state, self.parent, self.explored)
                found = True
                return True
        if found == False:
            return False


    def action_IDS(self, lst, node_, depth, limit):
        if (not lst in self.explored):
            if (lst[2] >= 1):   # Allowed Movement should be greater than or equall to one
                self.explored[lst] = True
                self.distance[lst] = self.distance[node_] + 1
                self.parent[lst] = node_
                if(self.deepen(lst, depth + 1, limit) == True):
                    return True
        
        elif(self.distance[lst] > self.distance[node_] + 1): # distance is shorter through this parent
            self.distance[lst] = self.distance[node_] + 1
            self.parent[lst] = node_
            if(self.deepen(lst, depth + 1, limit) == True):
                return True
        return False
    


    def deepen(self, node, depth, limit):
        if (depth == limit):
            return False
    
        node_pos, x, y, node_enemy_area, node_ally_index, node_ally_string = self.elicit_data(node)
        
        # Break condition
        if node == self.goal_state:
            return True
        
        child = list(copy.deepcopy(node))
        child = self.take_or_put_ally(child, node_ally_index, node_ally_string, x, y)
            
        if (x - 1 >= 0):
            if (self.table[x - 1][y][0] != '@'):
                lst = self.make_new_child(x - 1, y, node_enemy_area, child)
                if(self.action_IDS(lst, node, depth, limit) == True):
                    return True

        if(y - 1 >= 0):
            if (self.table[x][y - 1][0] != '@'):
                lst = self.make_new_child(x, y - 1, node_enemy_area, child)
                if(self.action_IDS(lst, node, depth, limit) == True):
                    return True

        if(x + 1 <= self.rows - 1):
            if(self.table[x + 1][y][0] != '@'):
                lst = self.make_new_child(x + 1, y, node_enemy_area, child)
                if(self.action_IDS(lst, node, depth, limit) == True):
                    return True
                    
        if(y + 1 <= self.columns - 1):
            if (self.table[x][y + 1][0] != '@'):
                lst = self.make_new_child(x, y + 1, node_enemy_area, child)
                if(self.action_IDS(lst, node, depth, limit) == True):
                    return True
        
        return False


## A* Algorithm (heuristic)

Each node in this algorithm has a cost f(node) = g(node) + h(node). g is the real cost of reaching that node from start and h is a heuristic function that estimates the remained path cost from that node to goal.

The frontier set maintains each node with its f() cost and in each step, node with the lowest cost is popped and explored.(run_Astar function)

The key point in this algorithm is calculating f for the childs of that node. In this way, at first we should obtain the nodes's real cost, so we subtract the h cost from f cost for that node. This cost is incremented by one and is the child's real cost (since each edge has cost one.). The heuristric cost is then added and finally the f cost of child is calculated.

Since we are traversing the explored set and alot of states should be checked (the heuristic is far less than real cost), we use set data structure for that. 

In [5]:


class Astar(Basic):
    def __init__(self, file_name, alpha_):
        if(alpha_ == 1):
            print("Running non-weghited Astar on file %s" %file_name)
        else:
            print("Running weighted Astar with coefficient equal to %s on file %s" %(alpha_, file_name))
        super().__init__(file_name)
        self.explored = set([])
        self.frontier = []
        self.previously_explored = dict()
        self.parent = dict()
        self.alpha = alpha_
        

    def hueristic(self,state):
        if(state[3] != '-1'):
            return self.alpha
        else:
            return 0

    def action_Astar(self, lst, fn, node_):
        if(not lst in self.explored):
            if(not (fn, lst) in self.previously_explored):
                if(lst[2] >= 1):
                    self.parent[lst] = node_
                    heappush(self.frontier, (fn, lst))
                    self.previously_explored[(fn, lst)] = 1       


    def run_Astar(self):
        heappush(self.frontier, (0 + self.hueristic(self.start_state), self.start_state))
        self.previously_explored[(0 + self.hueristic(self.start_state), self.start_state)] = 1

        while (self.frontier):
            data = heappop(self.frontier)
            node = data[1]
            #print(node)
            node_fn = data[0]
            node_pos, x, y, node_enemy_area, node_ally_index, node_ally_string = self.elicit_data(node)
        
            
            if node == self.goal_state:
                self.end_search(node, self.parent, self.explored)
                return True
            
            
            self.explored.add(node)

            child = list(copy.deepcopy(node))
            child = self.take_or_put_ally(child, node_ally_index, node_ally_string, x, y)

            child_real_cost = (node_fn - self.hueristic(node)) # real cost of parent
            child_real_cost += 1 # real cost of child
            child_fn = child_real_cost + self.hueristic(child)


            if (x - 1 >= 0):
                if (self.table[x - 1][y][0] != '@'):
                    lst = self.make_new_child(x - 1, y, node_enemy_area, child)
                    self.action_Astar(lst, child_fn, node)

            if(y - 1 >= 0):
                if (self.table[x][y - 1][0] != '@'):
                    lst = self.make_new_child(x, y - 1, node_enemy_area, child)
                    self.action_Astar(lst, child_fn, node)

            if(x + 1 <= self.rows - 1):
                if(self.table[x + 1][y][0] != '@'):
                    lst = self.make_new_child(x + 1, y, node_enemy_area, child)
                    self.action_Astar(lst, child_fn, node)  

            if(y + 1 <= self.columns - 1):
                if (self.table[x][y + 1][0] != '@'):
                    lst = self.make_new_child(x, y + 1, node_enemy_area, child)
                    self.action_Astar(lst, child_fn, node)

        return False
    

## Proof of consistency of heuristic

As we said, the heuristic should be consistent to reach the optimal solution. The hueristic which I chose for each state is number of allies whom Gandalf is carrying. since in each step we can carry at most one ally, this value is 0 or 1. 

Now we want to proof the consistency:
For each pair of nodes (p1,p2) we have:

                     abs(h(p1) - h(p2)) <= the real cost of reaching p1 from p2
                     
The above statement is obvious since the left side gets at most one and in reality, cost of reaching another node from our current position is at least one (edges have cost).                       

## Weigthed A*
Weighted A* is simply just multipling the heuristic cost by an coefficient (alpha). Why do we do that? Because the more the heuristic gets closed to real cost, the less states are explored and our algorithm runs more efficient. As said in the description, A* should be run three times, one for non-weighted and two times with different coefficients.   

Ofcourse it should be mentioned that if the coefficient gets extremely large, the algorithm isn't optimal any more, because f() cost gets higher than actual cost.

Some functions are defined to run the above algorithms on tests 

In [6]:
def run_BFS(file_name):
    start = time.time()
    b = BFS(file_name)
    b.print_table()
    found = b.run_BFS()
    if found == False:
        print("Goal isn't found!")
    finish = time.time()
    print("Time spent: %s \n\n\n" %(finish - start))
            
        
        
def run_IDS(file_name):
    start = time.time()
    depth = 100    
    d = IDS(file_name, depth)
    result = d.run_IDS()
    if result == False:
         print("Goal is placed deeper than %s" %(depth))
    finish = time.time()
    print("Time spent: %s \n\n\n" %(finish - start))
            
        
        
def run_Astar(file_name, alpha_):
    start = time.time()
    B = Astar(file_name, alpha_)
    result = B.run_Astar()
    if result == False:
        print("Goal is'nt found!")
    finish = time.time()
    print("Time spent: %s \n\n\n" %(finish - start))
    

In [7]:
run_BFS("test_00.txt")
run_IDS("test_00.txt")
run_Astar("test_00.txt", 1)   # Unweighted Astar
run_Astar("test_00.txt", 2)
run_Astar("test_00.txt", 3)



Running BFS algorithm on file test_00.txt
[['-'], ['-'], ['-'], ['-'], ['-'], ['-', '1'], ['B'], ['-', '&1'], ['-']]
[['-'], ['-'], ['-'], ['A'], ['-'], ['B'], ['@'], ['B'], ['-']]
[['-'], ['-', '0'], ['A'], ['@'], ['A'], ['-', '&0'], ['B'], ['-'], ['-']]
Goal found !
Path through start to goal: DDRURRRDRUURRDDR
Length of the answer: 16
Number of states seen: 189
Time spent: 0.0024330615997314453 



Running IDS algorithm on file test_00.txt
Goal is found in depth 17 !
Goal found !
Path through start to goal: DDRURRRDRUURRDDR
Length of the answer: 16
Number of states seen: 179
Time spent: 0.01989603042602539 



Running non-weghited Astar on file test_00.txt
Goal found !
Path through start to goal: RDDURRRRDUURRRDD
Length of the answer: 16
Number of states seen: 166
Time spent: 0.0021734237670898438 



Running weighted Astar with coefficient equal to 2 on file test_00.txt
Goal found !
Path through start to goal: RDDURRRRDUURRRDD
Length of the answer: 16
Number of states seen: 158
Time

In [8]:
run_BFS("test_01.txt")
run_IDS("test_01.txt")
run_Astar("test_01.txt", 1)   # Unweighted Astar
run_Astar("test_01.txt", 2)
run_Astar("test_01.txt", 3)



Running BFS algorithm on file test_01.txt
[['-'], ['-'], ['-'], ['A'], ['-'], ['-'], ['B'], ['B'], ['B']]
[['-'], ['C'], ['A'], ['@'], ['A'], ['B'], ['B'], ['@'], ['B']]
[['C'], ['@'], ['C'], ['A'], ['-', '0'], ['-'], ['B'], ['B'], ['B']]
[['-'], ['C'], ['-'], ['-'], ['-'], ['-'], ['-', '1'], ['B'], ['-']]
[['-'], ['-'], ['-'], ['D'], ['-'], ['-'], ['F'], ['-'], ['-', '2']]
[['-'], ['E'], ['D'], ['@'], ['D'], ['F'], ['F'], ['F'], ['-']]
[['E'], ['@'], ['E'], ['D'], ['F'], ['F'], ['@'], ['F'], ['F']]
[['-'], ['E'], ['-'], ['-'], ['-'], ['F'], ['F'], ['F'], ['-']]
[['-'], ['-', '&0'], ['-'], ['-', '&1'], ['-'], ['-', '&2'], ['F'], ['-'], ['-']]
Goal found !
Path through start to goal: DRRDRRLLDDDDDLDURUUUURRRRLLDDDLDDUURUURRRRDDDLDLLRRR
Length of the answer: 52
Number of states seen: 2097
Time spent: 0.018826723098754883 



Running IDS algorithm on file test_01.txt
Goal is found in depth 53 !
Goal found !
Path through start to goal: DRRDRRLLDDDDDLDURUUUURRRRLLDDDLDDUURUURRRRDDDLDLLRRR
L

In [9]:
run_BFS("test_02.txt")
run_IDS("test_02.txt")
run_Astar("test_02.txt", 1)   # Unweighted Astar
run_Astar("test_02.txt", 2)
run_Astar("test_02.txt", 3)

Running BFS algorithm on file test_02.txt
[['-'], ['-'], ['-'], ['-'], ['-'], ['-'], ['-'], ['-', '0']]
[['-'], ['A'], ['-'], ['-'], ['-'], ['-'], ['B'], ['-']]
[['A'], ['A'], ['A'], ['-'], ['-'], ['B'], ['@'], ['B']]
[['A'], ['@'], ['A'], ['A'], ['-'], ['-'], ['B'], ['-']]
[['A'], ['A'], ['A'], ['-'], ['C'], ['-'], ['-'], ['-']]
[['-', '&0'], ['A'], ['-'], ['C'], ['C'], ['C'], ['-', '&1'], ['-']]
[['-'], ['-'], ['C'], ['C'], ['@'], ['C'], ['C'], ['-']]
[['-'], ['-', '1'], ['-'], ['C'], ['C'], ['C'], ['-'], ['-']]
Goal found !
Path through start to goal: RRRRRRRLLLLDDDDLDLLDDRUURURRRDRDDR
Length of the answer: 34
Number of states seen: 456
Time spent: 0.015552759170532227 



Running IDS algorithm on file test_02.txt
Goal is found in depth 37 !
Goal found !
Path through start to goal: RRRRRRRLLLLDDDDLDLLDDRUURURRRDRDDR
Length of the answer: 34
Number of states seen: 450
Time spent: 0.18784189224243164 



Running non-weghited Astar on file test_02.txt
Goal found !
Path through start to

In [10]:
run_BFS("test_03.txt")
run_IDS("test_03.txt")
run_Astar("test_03.txt", 1)   # Unweighted Astar
run_Astar("test_03.txt", 2)
run_Astar("test_03.txt", 3)

Running BFS algorithm on file test_03.txt
[['-'], ['-'], ['-'], ['-'], ['A'], ['-'], ['-'], ['-', '0'], ['-'], ['-', '&1']]
[['-'], ['-'], ['-'], ['A'], ['A'], ['A'], ['-'], ['-'], ['B'], ['-']]
[['C'], ['-'], ['A'], ['A'], ['@'], ['A'], ['A'], ['B'], ['@'], ['B']]
[['C'], ['C'], ['-', '3'], ['A'], ['A'], ['A'], ['-'], ['-'], ['B'], ['-']]
[['@'], ['C'], ['C'], ['-'], ['A'], ['-', '2'], ['-'], ['F'], ['-'], ['-', '&2']]
[['C'], ['C'], ['-'], ['D'], ['-'], ['-'], ['F'], ['F'], ['F'], ['-']]
[['C'], ['E'], ['D'], ['@'], ['D'], ['F'], ['F'], ['F'], ['F'], ['F']]
[['E'], ['@'], ['E'], ['D'], ['F'], ['F'], ['F'], ['@'], ['F'], ['F']]
[['-', '&3'], ['E'], ['-'], ['-'], ['-'], ['F'], ['F'], ['F'], ['F'], ['F']]
[['-'], ['-', '1', '&0'], ['-'], ['-'], ['-'], ['-'], ['F'], ['F'], ['F'], ['-']]
Goal found !
Path through start to goal: DDRDRDDLDLDDDRURUUUUUUUURRRRRRRLLLLLLLDDDDDDDDLDURUUUURRRRRRRDDDDD
Length of the answer: 66
Number of states seen: 7362
Time spent: 0.06037402153015137 



Running

## Concluding the algorithm results in neat tables


### test_00.txt
| --- | Length of answer | Satates seen | Time spent(average)|
| --- | --- | --- | --- |
| BFS | 48 | 12499 | 0.1s |
| IDS | 48 | 12023 | 1.7s |
| non-weighted A* | 48 | 10348 | 0.08s |
| weighted A*(2) | 48 | 9230 | 0.07s |
| weighted A*(3) | 48 | 8140 | 0.06s |



### test_01.txt
| --- | Length of answer | Satates seen | Time spent(average)|
| --- | --- | --- | --- |
| BFS | 52 | 2097 | 0.01s |
| IDS | 52 | 2094 | 2.2s |
| non-weighted A* | 52 | 2074 | 0.02s |
| weighted A*(2) | 52 | 2050 | 0.02s |
| weighted A*(3) | 52 | 2005 | 0.01s |


### test_02.txt
| --- | Length of answer | Satates seen | Time spent(average)|
| --- | --- | --- | --- |
| BFS | 34 | 456 | 0.005s |
| IDS | 34 | 450 | 0.2s |
| non-weighted A* | 34 | 418 | 0.003s |
| weighted A*(2) | 34 | 396 | 0.003s |
| weighted A*(3) | 34 | 377 | 0.003s |


### test_03.txt
| --- | Length of answer | Satates seen | Time spent(average)|
| --- | --- | --- | --- |
| BFS | 66 | 7362 | 0.06s |
| IDS | 66 | 7274 | 9.1s |
| non-weighted A* | 66 | 7244 | 0.06s |
| weighted A*(2) | 66 | 7097 | 0.06s |
| weighted A*(3) | 66 | 6944 | 0.06s |


## Cons and Pros of algorithm compared to each other
All three algorithms mentioned are complete and optimal(A* depends on its heuristic), but here are some detailed differences between them.

At first, some parameters are defined:

b: branching factor of graph

d: depth of goal 

### BFS:

Time complexity: O(b^d)

Space complexity: O(b^d) 

#### pros:

Doesn't handle duplicated states.

#### cons:

High memory consumption because it stores all nodes in a level and then goes to next level.

If the goal is located in deep levels and far from root, it wastes time by proceeding level by level.

### IDS:

Time complexity:

O((d + 1)b^0 + db^1 + (d-1)b^2 + ... + b^d) = O(b^d)

Space complexity: O(bm)

#### pros:

Takes advantage of both BFS and DFS strategies.

Less memory consumption, since it goes down depth-wise (like DFS).

#### cons:

handles duplicated states and wastes time by starting again from root in each iteration.

### A*:

Time and memory complexity:

Each node that has f() cost less than or equal to the optimal path cost is explored.
(The h function plays an important rule).

#### pros:
If heuristic function is consistent, we don't need to check duplicated states.(otherwise, we should)


#### cons:
If hueristic is not addmissble, it doesn't generate optimal path.

If f() cost (made up of hueristic and real cost) is far less than actual value, it explores alot of states and wastes time and memory.(That's why we apply the weighted A* technique to multiply hueristics and make f cost closer to real cost)   

## Conclusion

We have learn't about some informed and uninformed search methods and how they work.

Further more, we learn't that using a heuristic method and improving it is another great method along side other methods. 

At last, we understood that which one gives us the optimal path from start to goal and their advantages and disadvantages compared to each other.

